## Assigment 6
Karl-Axel Jönsson BME4

In [2]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #To get rid of annoying tf messages
from matplotlib import pyplot as plt
import cinc2022metrics as cm # Our own little metrics file
from tensorflow import keras
from keras import utils
from sklearn.model_selection import train_test_split

In [3]:
feats = pd.read_csv('/Users/karl-axeljonsson/Downloads/feats.csv', header=None, usecols = [i+1 for i in range(22)]) 
murmur_labels = pd.read_csv('/Users/karl-axeljonsson/Downloads/murmur_labels.csv', header=None)
outcome_labels = pd.read_csv('/Users/karl-axeljonsson/Downloads/outcome_labels.csv', header=None)

feats = feats.to_numpy()
murmur_labels = murmur_labels.to_numpy().ravel()
outcome_labels = outcome_labels.to_numpy().ravel()

In [4]:

X_train,X_test,y_train_murmur,y_test_murmur,y_train_outcome,y_test_outcome=train_test_split(feats, murmur_labels, outcome_labels, test_size=0.2,random_state=2)
murmur_classes=['Present', 'Unknown', 'Absent']
outcome_classes=['Abnormal', 'Normal']


In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression



## GradientBoosting

In [21]:
clf=GradientBoostingClassifier(n_estimators=30,learning_rate=0.1).fit(X_train,y_train_murmur)
y_hat_murmur=clf.predict_proba(X_test)
clf_2=GradientBoostingClassifier(n_estimators=50, learning_rate=0.2, max_leaf_nodes=10).fit(X_train, y_train_outcome)
y_hat_outcome=clf_2.predict_proba(X_test)

y_test_murmur_bin=utils.to_categorical(y_test_murmur)
y_test_outcome_bin=utils.to_categorical(y_test_outcome)



murmur_scores, outcome_scores = cm.compute_scores(y_test_murmur_bin,   # One-hot encoded test labels for murmur eg. [1 0 0]
                                                  y_hat_murmur,  # One-hot encoded predicted probabilities for murmur eg. [0.1 0.7 0.2]                             
                                                  murmur_classes,      # As defined before
                                                  y_test_outcome_bin,  # One-hot encoded test labels for outcome eg. [1 0]
                                                  y_hat_outcome, # One-hot encoded predicted probabilities for outcome eg. [0.1 0.9]
                                                  outcome_classes)     # As defined before

cm.print_scores(murmur_scores, outcome_scores)


#Murmur scores
AUROC,AUPRC,F-meas,Accuracy,Weighted Accuracy,Cost
0.557,0.376,0.299,0.735,0.673,14280.360

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.521,0.520,0.517,0.519,0.503,14648.111

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.590,0.552,0.530
AUPRC,0.282,0.785,0.060
F-measure,0.050,0.847,0.000
Accuracy,0.026,0.979,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.521,0.521
AUPRC,0.460,0.580
F-measure,0.492,0.543
Accuracy,0.494,0.540



## Bagging

In [27]:
clf=BaggingClassifier(KNeighborsClassifier(3),max_features=1, max_samples=5).fit(X_train, y_train_murmur)
y_hat_murmur=clf.predict_proba(X_test)
clf_2=BaggingClassifier(KNeighborsClassifier(2),max_features=2).fit(X_train,y_train_outcome)
y_hat_outcome=clf_2.predict_proba(X_test)


murmur_scores, outcome_scores = cm.compute_scores(y_test_murmur_bin,   # One-hot encoded test labels for murmur eg. [1 0 0]
                                                  y_hat_murmur,  # One-hot encoded predicted probabilities for murmur eg. [0.1 0.7 0.2]                             
                                                  murmur_classes,      # As defined before
                                                  y_test_outcome_bin,  # One-hot encoded test labels for outcome eg. [1 0]
                                                  y_hat_outcome, # One-hot encoded predicted probabilities for outcome eg. [0.1 0.9]
                                                  outcome_classes)     # As defined before

cm.print_scores(murmur_scores, outcome_scores)



#Murmur scores
AUROC,AUPRC,F-meas,Accuracy,Weighted Accuracy,Cost
0.465,0.315,0.301,0.672,0.621,14718.995

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.535,0.529,0.513,0.513,0.523,14167.836

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.415,0.416,0.566
AUPRC,0.175,0.707,0.062
F-measure,0.103,0.800,0.000
Accuracy,0.079,0.879,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.535,0.535
AUPRC,0.512,0.547
F-measure,0.505,0.521
Accuracy,0.528,0.500



## AdaBoost

In [23]:
clf=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=10).fit(X_train,y_train_murmur)
y_hat_murmur=clf.predict_proba(X_test)
clf_2=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=4).fit(X_train,y_train_outcome)
y_hat_outcome=clf_2.predict_proba(X_test)


murmur_scores, outcome_scores = cm.compute_scores(y_test_murmur_bin,   # One-hot encoded test labels for murmur eg. [1 0 0]
                                                  y_hat_murmur,  # One-hot encoded predicted probabilities for murmur eg. [0.1 0.7 0.2]                             
                                                  murmur_classes,      # As defined before
                                                  y_test_outcome_bin,  # One-hot encoded test labels for outcome eg. [1 0]
                                                  y_hat_outcome, # One-hot encoded predicted probabilities for outcome eg. [0.1 0.9]
                                                  outcome_classes)     # As defined before

cm.print_scores(murmur_scores, outcome_scores)



#Murmur scores
AUROC,AUPRC,F-meas,Accuracy,Weighted Accuracy,Cost
0.549,0.358,0.285,0.746,0.679,14718.995

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.536,0.538,0.496,0.513,0.662,11959.202

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.506,0.574,0.568
AUPRC,0.211,0.791,0.071
F-measure,0.000,0.855,0.000
Accuracy,0.000,1.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.536,0.536
AUPRC,0.497,0.579
F-measure,0.589,0.403
Accuracy,0.742,0.310



## Voting

In [37]:
clf1=RandomForestClassifier(n_estimators=50,random_state=1).fit(X_train,y_train_murmur)
clf2=LogisticRegression(random_state=1).fit(X_train,y_train_murmur)
clf3=KNeighborsClassifier().fit(X_train,y_train_murmur)

clf=VotingClassifier(estimators=[('rf', clf1), ('lr', clf2), ('knn', clf3)],voting='soft').fit(X_train,y_train_murmur)
y_hat_murmur=clf.predict_proba(X_test)

clf_2=VotingClassifier(estimators=[('rf', clf1), ('lr', clf2), ('knn', clf3)],voting='soft').fit(X_train,y_train_outcome)
y_hat_outcome=clf_2.predict_proba(X_test)


murmur_scores, outcome_scores = cm.compute_scores(y_test_murmur_bin,   # One-hot encoded test labels for murmur eg. [1 0 0]
                                                  y_hat_murmur,  # One-hot encoded predicted probabilities for murmur eg. [0.1 0.7 0.2]                             
                                                  murmur_classes,      # As defined before
                                                  y_test_outcome_bin,  # One-hot encoded test labels for outcome eg. [1 0]
                                                  y_hat_outcome, # One-hot encoded predicted probabilities for outcome eg. [0.1 0.9]
                                                  outcome_classes)     # As defined before

cm.print_scores(murmur_scores, outcome_scores)




#Murmur scores
AUROC,AUPRC,F-meas,Accuracy,Weighted Accuracy,Cost
0.417,0.348,0.283,0.735,0.669,14718.995

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.488,0.503,0.504,0.508,0.470,15392.204

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.479,0.449,0.322
AUPRC,0.195,0.708,0.139
F-measure,0.000,0.848,0.000
Accuracy,0.000,0.986,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.488,0.488
AUPRC,0.467,0.538
F-measure,0.462,0.546
Accuracy,0.449,0.560

hej


### Conclusion
The best weighted accuracy received was from AdaBoost or bagging, approx 0.1 from the best result in the competition. The cost was drastically lowest in AdaBoost than any other of the other ones with only 800 from the lowest value in the competition.